In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
import re
import pandas as pd
import matplotlib.pyplot as plt
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from unidecode import unidecode

In [2]:
# Set up search parameters
min_price = 1000
max_price = 30000
days_listed = 7
min_mileage = 50000
max_mileage = 200000
min_year = 2000
max_year = 2020
transmission = "automatic"
query = "Honda Civic"
#"102170323157613" = Moncton NB, "114723638540069" = Timmins ON, "111551465530472" = Thunder Bay ON
cities = ["102170323157613", "montreal", "toronto", "114723638540069", "111551465530472", "winnipeg", "regina", "calgary", "vancouver"]

In [3]:
browser = webdriver.Chrome()

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 114
Current browser version is 118.0.5993.90 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0060A813+48355]
	(No symbol) [0x0059C4B1]
	(No symbol) [0x004A5358]
	(No symbol) [0x004C61AC]
	(No symbol) [0x004C1EF3]
	(No symbol) [0x004C0579]
	(No symbol) [0x004F0C55]
	(No symbol) [0x004F093C]
	(No symbol) [0x004EA536]
	(No symbol) [0x004C82DC]
	(No symbol) [0x004C93DD]
	GetHandleVerifier [0x0086AABD+2539405]
	GetHandleVerifier [0x008AA78F+2800735]
	GetHandleVerifier [0x008A456C+2775612]
	GetHandleVerifier [0x006951E0+616112]
	(No symbol) [0x005A5F8C]
	(No symbol) [0x005A2328]
	(No symbol) [0x005A240B]
	(No symbol) [0x00594FF7]
	BaseThreadInitThunk [0x76B6FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x76FD7C6E+286]
	RtlGetAppContainerNamedObjectPath [0x76FD7C3E+238]


In [4]:
# Create a list to store htmls
soups = []

for i, city in enumerate(cities):
    # Set up base url
    base_url = f'https://www.facebook.com/marketplace/{city}/search?'
    #Set up full url
    url = f"{base_url}minPrice={min_price}&maxPrice={max_price}&daysSinceListed={days_listed}&maxMileage={max_mileage}&maxYear={max_year}&minMileage={min_mileage}&minYear={min_year}&transmissionType={transmission}&query={query}&exact=false"

    # Visit the website
    browser.get(url)

    # Find the div element by its CSS selector
    wait = WebDriverWait(browser, 10)
    div_element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'div.x1i10hfl.x1qjc9v5.xjbqb8w.xjqpnuy.xa49m3k.xqeqjp1.x2hbi6w.x13fuv20.xu3j5b3.x1q0q8m5.x26u7qi.x972fbf.xcfux6l.x1qhh985.xm0m39n.x9f619.x1ypdohk.xdl72j9.x2lah0s.xe8uvvx.x11i5rnm.xat24cr.x1mh8g0r.x2lwn1j.xeuugli.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x1n2onr6.x16tdsg8.x1hl2dhg.xggy1nq.x1ja2u2z.x1t137rt.x1o1ewxj.x3x9cwd.x1e5q0jg.x13rtm0m.x1q0g3np.x87ps6o.x1lku1pv.x78zum5.x1a2a7pz.x1xmf6yo[tabindex="0"]')))

    # Click on the div element
    div_element.click()

    wait = WebDriverWait(browser, 10)

    # Wait for the label element to be clickable
    wait = WebDriverWait(browser, 10)
    
    # Account for variation in label element containing search radius (km)
    try:
        label_element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'label[for=":rj:"]')))
    except:
        try:
            label_element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'label[for=":re:"]')))
        except:
            label_element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'label[for=":rd:"]')))

    # Click on the label element
    label_element.click()

    # Press the "Down" key 11 times
    for _ in range(12):
        label_element.send_keys(Keys.DOWN)

    # Press Enter
    label_element.send_keys(Keys.ENTER)

    # Wait for the "Apply" button to be clickable
    wait = WebDriverWait(browser, 10)
    apply_div = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'div[aria-label="Apply"]')))

    # Click on the div element
    apply_div.click()
    
    # Scroll down to load more results
    scroll_count = 3
    scroll_delay = 1

    for _ in range(scroll_count):
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_delay)
    
    # Parse the HTML
    html = browser.page_source
    
    # Create a BeautifulSoup object from the scraped HTML
    soups.append(soup(html, 'html.parser'))
    
    # Check if HTML was scraped correctly
    print(f'finished {city}')
    print('--------------------------------')

finished 102170323157613
--------------------------------
finished montreal
--------------------------------
finished toronto
--------------------------------
finished 114723638540069
--------------------------------
finished 111551465530472
--------------------------------
finished winnipeg
--------------------------------
finished regina
--------------------------------
finished calgary
--------------------------------
finished vancouver
--------------------------------


In [5]:
# Close the browser
browser.quit()

In [7]:
# Create empty lists to store the extracted information
titles_list = []
prices_list = []
mileage_list = []
urls_list = []

# Iterate over the soups
for soup in soups:
    # Extract title information
    titles_div = soup.find_all('span', class_="x1lliihq x6ikm8r x10wlt62 x1n2onr6")
    titles_list.extend([title.text.strip() for title in titles_div])
    
    # Extract price information
    prices_div = soup.find_all('span', class_="x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x xudqn12 x676frb x1lkfr7t x1lbecb7 x1s688f xzsf02u")
    prices_list.extend([price.text.strip() for price in prices_div])
    
    # Extract mileage information
    mileage_div = soup.find_all('span', class_="x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x4zkp8e x3x7a5m x1nxh6w3 x1sibtaa xo1l8bm xi81zsa")
    mileage_list.extend([mileage.text.strip() for mileage in mileage_div])
    
    # Extract URL information
    urls_div = soup.find_all('a', class_="x1i10hfl xjbqb8w x6umtig x1b1mbwd xaqea5y xav7gou x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz x1heor9g x1lku1pv")
    urls_list.extend([url.get('href') for url in urls_div])

In [17]:
# # Extracted mileage list (separate from location and extract numeric values only)
# mileage_pattern_km = r'(\d+)K km'
# mileage_pattern_miles = r'(\d+)K miles'

# # Extract mileage according to pattern and transform values into integers
# mileage_clean = []
# location = []
# for item in mileage_list:
#     match_mileage_km = re.search(mileage_pattern_km, item)
#     match_mileage_miles = re.search(mileage_pattern_miles, item)
#     if match_mileage_km or match_mileage_miles:
#         if match_mileage_km:
#             mileage_clean.append(int(match_mileage_km.group(1)) * 1000)
#         else:
#             mileage_clean.append(int(match_mileage_miles.group(1))*1000)
#     # Account for empty fields
#     elif item == "":
#         mileage_clean.append(0) 
            
# len(mileage_clean)

463

In [113]:
# Set patterns for mileage and location
mileage_pattern_km = r'(\d+)K km'
mileage_pattern_miles = r'(\d+)K miles'
location_pattern = r'\b[\w\s-]+(?:, \w{2})\b'

# Extract mileage, city and province according to pattern and transform values into integers
mileage_clean = []
city_data = []
province_data = []

# Iterate through mileage list to separate location from mileage
for item in mileage_list:
    match_mileage_km = re.search(mileage_pattern_km, item)
    match_mileage_miles = re.search(mileage_pattern_miles, item)
    match_location = re.search(location_pattern, item)

    # Check for mileage pattern using regex
    if match_mileage_km or match_mileage_miles:
        if match_mileage_km:
            mileage_clean.append(int(match_mileage_km.group(1)) * 1000)
        else:
            mileage_clean.append(int(match_mileage_miles.group(1)) * 1000)
            
    # Check for location pattern using regex     
    elif match_location:
        location = match_location.group(0)
        city, province = map(str.strip, location.split(','))
        city_data.append(city)
        province_data.append(province)
        
    elif item == "":
        mileage_clean.append(0)

# Check for equal length on all lists        
print(len(city_data), len(province_data), len(mileage_clean))
city_data

463 463 463


['Moncton',
 'Moncton',
 'Moncton',
 'Moncton',
 'Riverview',
 'Amherst',
 'Moncton',
 'Amherst',
 'Amherst',
 'Prince',
 'Terrebonne',
 'Montréal',
 'Laval',
 'Montréal',
 'Montréal',
 'Mont-Royal',
 'Montréal',
 'Montréal-Est',
 'Montréal',
 'Montréal',
 'Montréal',
 'Montréal',
 'Montréal',
 'Assomption',
 'Montréal',
 'Montréal',
 'Montréal',
 'Laval',
 'Montréal',
 'Laval',
 'Montréal',
 'Laval',
 'Laval',
 'Montréal',
 'Montréal',
 'Blainville',
 'Montréal',
 'Ste-Julie',
 'Mirabel',
 'Montréal',
 'Montréal',
 'Montréal',
 'St-Constant',
 'Montréal',
 'Longueuil',
 'Montréal',
 'Laval',
 'Laval',
 'Longueuil',
 'Montréal',
 'Montréal',
 'Montréal',
 'Montréal',
 'Dollard-des Ormeaux',
 'Montréal',
 'Longueuil',
 'Mirabel',
 'Laval',
 'Châteauguay',
 'Laval',
 'Montréal',
 'Montréal',
 'Montréal',
 'Montréal',
 'Blainville',
 'Laval',
 'Montréal',
 'Laval',
 'Montréal-Est',
 'Montréal',
 'Laval',
 'Montréal',
 'Montréal',
 'Montréal',
 'Ste-Sophie',
 'Montréal',
 'Terrasse-Vaudreu

In [114]:
# Add all values to a list of dictionaries
vehicles_list = []

for i, item in enumerate(titles_list):
    cars_dict = {}
    
    title_split = titles_list[i].split()
    
    cars_dict["City"] = city_data[i]
    cars_dict["Province"] = province_data[i]
    cars_dict["Year"] = int(title_split[0])
    cars_dict["Make"] = title_split[1]
    cars_dict["Model"] = title_split[2]
    cars_dict["Price"] = int(re.sub(r'[^\d.]', '', prices_list[i]))
    cars_dict["Mileage"] = mileage_clean[i]
    cars_dict["URL"] = urls_list[i]
    vehicles_list.append(cars_dict)
    
print(vehicles_list)

[{'City': 'Moncton', 'Province': 'NB', 'Year': 2010, 'Make': 'Honda', 'Model': 'civic', 'Price': 4500, 'Mileage': 170000, 'URL': '/marketplace/item/790927115981218/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD'}, {'City': 'Moncton', 'Province': 'NB', 'Year': 2016, 'Make': 'Honda', 'Model': 'civic', 'Price': 16500, 'Mileage': 176000, 'URL': '/marketplace/item/550591577282477/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD'}, {'City': 'Moncton', 'Province': 'NB', 'Year': 2012, 'Make': 'Honda', 'Model': 'civic', 'Price': 12000, 'Mileage': 198000, 'URL': '/marketplace/item/7066306713384948/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD'}, {'City': 'Moncton', 'Province': 'NB', 'Year': 2012, 'Make': 'Honda', 'Model': 'civic', 'Price': 13000, 'Mileage': 102000, 'URL': '/marketplace/item/1449911362421633/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD'}, {'City': 'Riverview', 'Province': 'NB', 'Year': 2015, 

In [115]:
# Convert to DataFrame
vehicle_df = pd.DataFrame(vehicles_list)
vehicle_df.tail()

,City,Province,Year,Make,Model,Price,Mileage,URL
458,Surrey,BC,2006,Honda,civic,6500,192000,/marketplace/item/793335559041319/?ref=search&...
459,Surrey,BC,2014,Honda,civic,16899,132000,/marketplace/item/1114999312606400/?ref=search...
460,Langley,BC,2001,Honda,civic,3900,153000,/marketplace/item/1271587260125919/?ref=search...
461,Vancouver,BC,2017,Honda,civic,27995,52000,/marketplace/item/1305781527021832/?ref=search...
462,Vancouver,BC,2016,Honda,civic,23888,102000,/marketplace/item/484475217177138/?ref=search&...


In [116]:
# Drop duplicates based on URL
vehicle_df = vehicle_df.drop_duplicates(subset='URL')
vehicle_df.info()
vehicle_df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 359 entries, 0 to 462
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   City      359 non-null    object
 1   Province  359 non-null    object
 2   Year      359 non-null    int64 
 3   Make      359 non-null    object
 4   Model     359 non-null    object
 5   Price     359 non-null    int64 
 6   Mileage   359 non-null    int64 
 7   URL       359 non-null    object
dtypes: int64(3), object(5)
memory usage: 25.2+ KB


,City,Province,Year,Make,Model,Price,Mileage,URL
0,Moncton,NB,2010,Honda,civic,4500,170000,/marketplace/item/790927115981218/?ref=search&...
1,Moncton,NB,2016,Honda,civic,16500,176000,/marketplace/item/550591577282477/?ref=search&...
2,Moncton,NB,2012,Honda,civic,12000,198000,/marketplace/item/7066306713384948/?ref=search...
3,Moncton,NB,2012,Honda,civic,13000,102000,/marketplace/item/1449911362421633/?ref=search...
4,Riverview,NB,2015,Honda,civic,19999,112000,/marketplace/item/1362966634561957/?ref=search...


In [117]:
# Drop entries with where Mileage = 0
vehicle_df = vehicle_df[vehicles_df['Mileage'] != 0]

# Reset the index
vehicle_df = vehicle_df.reset_index(drop=True)

# Print information about the cleaned DataFrame
vehicle_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 357 entries, 0 to 356
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   City      357 non-null    object
 1   Province  357 non-null    object
 2   Year      357 non-null    int64 
 3   Make      357 non-null    object
 4   Model     357 non-null    object
 5   Price     357 non-null    int64 
 6   Mileage   357 non-null    int64 
 7   URL       357 non-null    object
dtypes: int64(3), object(5)
memory usage: 22.4+ KB


C:\Users\Victor\AppData\Local\Temp\ipykernel_31084\1883717805.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  vehicle_df = vehicle_df[vehicles_df['Mileage'] != 0]


In [118]:
vehicle_df.head()

,City,Province,Year,Make,Model,Price,Mileage,URL
0,Moncton,NB,2010,Honda,civic,4500,170000,/marketplace/item/790927115981218/?ref=search&...
1,Moncton,NB,2016,Honda,civic,16500,176000,/marketplace/item/550591577282477/?ref=search&...
2,Moncton,NB,2012,Honda,civic,12000,198000,/marketplace/item/7066306713384948/?ref=search...
3,Moncton,NB,2012,Honda,civic,13000,102000,/marketplace/item/1449911362421633/?ref=search...
4,Riverview,NB,2015,Honda,civic,19999,112000,/marketplace/item/1362966634561957/?ref=search...


In [119]:
#Remove vehicles different from the model on the initial query
model = query.split()[1]

vehicle_df_clean = vehicle_df[vehicle_df['Model'].str.lower() == model.lower()]

vehicle_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 298 entries, 0 to 356
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   City      298 non-null    object
 1   Province  298 non-null    object
 2   Year      298 non-null    int64 
 3   Make      298 non-null    object
 4   Model     298 non-null    object
 5   Price     298 non-null    int64 
 6   Mileage   298 non-null    int64 
 7   URL       298 non-null    object
dtypes: int64(3), object(5)
memory usage: 21.0+ KB


In [120]:
# Add "facebook.com" to the URLs
vehicle_df_clean['URL'] = 'facebook.com' + vehicle_df_clean['URL']
vehicle_df_clean.head()

C:\Users\Victor\AppData\Local\Temp\ipykernel_31084\1409949132.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vehicle_df_clean['URL'] = 'facebook.com' + vehicle_df_clean['URL']


,City,Province,Year,Make,Model,Price,Mileage,URL
0,Moncton,NB,2010,Honda,civic,4500,170000,facebook.com/marketplace/item/790927115981218/...
1,Moncton,NB,2016,Honda,civic,16500,176000,facebook.com/marketplace/item/550591577282477/...
2,Moncton,NB,2012,Honda,civic,12000,198000,facebook.com/marketplace/item/7066306713384948...
3,Moncton,NB,2012,Honda,civic,13000,102000,facebook.com/marketplace/item/1449911362421633...
4,Riverview,NB,2015,Honda,civic,19999,112000,facebook.com/marketplace/item/1362966634561957...


In [121]:
# Remove special characters and replace with their normal versions
vehicle_df_clean['City'] = vehicle_df_clean['City'].apply(lambda x: unidecode(x))

vehicle_df_clean.head

C:\Users\Victor\AppData\Local\Temp\ipykernel_31084\22830476.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vehicle_df_clean['City'] = vehicle_df_clean['City'].apply(lambda x: unidecode(x))


<bound method NDFrame.head of           City Province  Year   Make  Model  Price  Mileage   
0      Moncton       NB  2010  Honda  civic   4500   170000  \
1      Moncton       NB  2016  Honda  civic  16500   176000   
2      Moncton       NB  2012  Honda  civic  12000   198000   
3      Moncton       NB  2012  Honda  civic  13000   102000   
4    Riverview       NB  2015  Honda  civic  19999   112000   
..         ...      ...   ...    ...    ...    ...      ...   
352     Surrey       BC  2006  Honda  civic   6500   192000   
353     Surrey       BC  2014  Honda  civic  16899   132000   
354    Langley       BC  2001  Honda  civic   3900   153000   
355  Vancouver       BC  2017  Honda  civic  27995    52000   
356  Vancouver       BC  2016  Honda  civic  23888   102000   

                                                   URL  
0    facebook.com/marketplace/item/790927115981218/...  
1    facebook.com/marketplace/item/550591577282477/...  
2    facebook.com/marketplace/item/7066306

In [123]:
# Create CSV file
vehicle_df_clean.to_csv('vehicles_clean.csv', header=True, index=False)

In [137]:
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

# Define the SQLAlchemy base class
Base = declarative_base()

# Define the SQLAlchemy base class
Base = declarative_base()

# Define the vehicle_listings table as a SQLAlchemy model
class VehicleListing(Base):
    __tablename__ = 'vehicle_listings'
    id = Column(Integer, primary_key=True)
    City = Column(String)
    Province = Column(String)
    Year = Column(Integer)
    Make = Column(String)
    Model = Column(String)
    Price = Column(Float)
    Mileage = Column(Integer)
    URL = Column(String)

# Create the SQLite database engine
engine = create_engine('sqlite:///vehicle2.db')

# Create the tables in the database
Base.metadata.create_all(engine)

# Create a session to interact with the database
Session = sessionmaker(bind=engine)
session = Session()

# Insert data into the table
for _, row in vehicle_df_clean.iterrows():
    vehicle = VehicleListing(City=row['City'], Province=row['Province'], Year=row['Year'], Make=row['Make'],
                             Model=row['Model'], Price=row['Price'], Mileage=row['Mileage'], URL=row['URL'])
    session.add(vehicle)


# Commit the changes
session.commit()

# Close the session
session.close()